# 🌐 Lección 2: HTTP Requests y Beautiful Soup Avanzado

## 🎯 Objetivos de Aprendizaje

Al finalizar esta lección, serás capaz de:
- ✅ Dominar el protocolo HTTP y sus métodos
- ✅ Manejar headers, cookies y sesiones avanzadas
- ✅ Trabajar con formularios y POST requests
- ✅ Implementar técnicas avanzadas de Beautiful Soup
- ✅ Manejar diferentes tipos de contenido y encodings
- ✅ Crear scrapers robustos para sitios complejos

---

## 1. Protocolo HTTP en Profundidad 🔍

HTTP (HyperText Transfer Protocol) es el fundamento de toda comunicación web. Para hacer web scraping avanzado, necesitas entender cómo funciona realmente.

### 🔄 Métodos HTTP Principales

| Método | Propósito | Idempotente | Cacheable | Tiene Body |
|--------|-----------|-------------|-----------|------------|
| **GET** | 📥 Obtener datos | ✅ | ✅ | ❌ |
| **POST** | 📤 Enviar datos | ❌ | ❌ | ✅ |
| **PUT** | 🔄 Actualizar/crear | ✅ | ❌ | ✅ |
| **DELETE** | 🗑️ Eliminar recurso | ✅ | ❌ | ❌ |
| **HEAD** | 📋 Solo headers | ✅ | ✅ | ❌ |
| **OPTIONS** | 🔧 Opciones disponibles | ✅ | ✅ | ❌ |

In [ ]:
import requests
import json
from pprint import pprint
from urllib.parse import urljoin, urlparse
import time

def explorar_metodos_http():
    """Demonstrar diferentes métodos HTTP"""
    print("🌐 EXPLORANDO MÉTODOS HTTP\n")
    print("═" * 60)
    
    # URL de ejemplo que acepta diferentes métodos
    base_url = "https://httpbin.org"
    
    # 1. GET Request básico
    print("\n1️⃣ GET Request:")
    print("-" * 30)
    
    response = requests.get(f"{base_url}/get", params={'param1': 'valor1', 'param2': 'valor2'})
    print(f"Status Code: {response.status_code}")
    print(f"URL final: {response.url}")
    
    data = response.json()
    print(f"Parámetros enviados: {data['args']}")
    print(f"Headers enviados: {len(data['headers'])} headers")
    
    # 2. POST Request con datos
    print("\n2️⃣ POST Request:")
    print("-" * 30)
    
    post_data = {
        'usuario': 'scraper_usuario',
        'email': 'usuario@ejemplo.com',
        'mensaje': 'Datos enviados via POST'
    }
    
    response = requests.post(f"{base_url}/post", data=post_data)
    print(f"Status Code: {response.status_code}")
    
    data = response.json()
    print(f"Datos enviados: {data['form']}")
    print(f"Content-Type: {data['headers'].get('Content-Type')}")
    
    # 3. HEAD Request (solo headers)
    print("\n3️⃣ HEAD Request:")
    print("-" * 30)
    
    response = requests.head("https://httpbin.org/html")
    print(f"Status Code: {response.status_code}")
    print(f"Content-Type: {response.headers.get('content-type')}")
    print(f"Content-Length: {response.headers.get('content-length')}")
    print(f"Body length: {len(response.content)} (debería ser 0)")
    
    # 4. PUT Request
    print("\n4️⃣ PUT Request:")
    print("-" * 30)
    
    put_data = {'recurso': 'actualizado', 'version': '2.0'}
    response = requests.put(f"{base_url}/put", json=put_data)
    
    data = response.json()
    print(f"Status Code: {response.status_code}")
    print(f"JSON enviado: {data['json']}")
    
    # 5. DELETE Request
    print("\n5️⃣ DELETE Request:")
    print("-" * 30)
    
    response = requests.delete(f"{base_url}/delete")
    print(f"Status Code: {response.status_code}")
    print(f"Método confirmado: {response.json()['url']}")
    
# Ejecutar demostración
explorar_metodos_http()

## 2. Headers HTTP: Tu Identidad en la Web 🕵️

Los headers HTTP son metadatos que acompañan cada request y response. Son cruciales para un scraping exitoso.

In [ ]:
def analizar_headers_http():
    """Análisis completo de headers HTTP"""
    print("🕵️ ANÁLISIS COMPLETO DE HEADERS HTTP\n")
    print("═" * 70)
    
    # Headers básicos vs avanzados
    headers_basicos = {
        'User-Agent': 'Python-requests/2.28.0'
    }
    
    headers_avanzados = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1'
    }
    
    # Comparar respuestas
    print("1️⃣ Comparación: Headers Básicos vs Avanzados")
    print("-" * 50)
    
    # Request con headers básicos
    response_basico = requests.get('https://httpbin.org/headers', headers=headers_basicos)
    print("\n📤 Headers enviados (básicos):")
    for key, value in headers_basicos.items():
        print(f"   {key}: {value[:50]}{'...' if len(value) > 50 else ''}")
    
    # Request con headers avanzados
    response_avanzado = requests.get('https://httpbin.org/headers', headers=headers_avanzados)
    print("\n📤 Headers enviados (avanzados):")
    for key, value in headers_avanzados.items():
        print(f"   {key}: {value[:50]}{'...' if len(value) > 50 else ''}")
    
    # Analizar headers de respuesta
    print("\n\n2️⃣ Análisis de Headers de Respuesta")
    print("-" * 50)
    
    response = requests.get('https://httpbin.org/response-headers?Content-Type=application/json&Custom-Header=valor-personalizado')
    
    print("📥 Headers recibidos importantes:")
    headers_importantes = [
        'content-type', 'content-length', 'server', 'date', 
        'cache-control', 'set-cookie', 'location', 'custom-header'
    ]
    
    for header in headers_importantes:
        valor = response.headers.get(header, 'No presente')
        print(f"   {header.title()}: {valor}")
    
    # Headers de seguridad
    print("\n\n3️⃣ Headers de Seguridad Comunes")
    print("-" * 50)
    
    security_headers = {
        'X-Frame-Options': 'Previene clickjacking',
        'X-Content-Type-Options': 'Previene MIME sniffing',
        'X-XSS-Protection': 'Protección XSS',
        'Strict-Transport-Security': 'Fuerza HTTPS',
        'Content-Security-Policy': 'Control de recursos',
        'Referrer-Policy': 'Control de referrer'
    }
    
    # Probar un sitio real para ver headers de seguridad
    try:
        response = requests.head('https://github.com', timeout=5)
        print("🔒 Headers de seguridad en GitHub:")
        
        for header, descripcion in security_headers.items():
            valor = response.headers.get(header.lower(), 'No presente')
            status = "✅" if valor != 'No presente' else "❌"
            print(f"   {status} {header}: {valor[:50]}{'...' if len(str(valor)) > 50 else ''}")
            
    except requests.RequestException as e:
        print(f"❌ Error al acceder a GitHub: {e}")
    
    # User-Agents más comunes
    print("\n\n4️⃣ User-Agents Recomendados para Scraping")
    print("-" * 50)
    
    user_agents = {
        'Chrome Windows': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Chrome Mac': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Firefox Windows': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0',
        'Safari Mac': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15',
        'Edge Windows': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0'
    }
    
    for navegador, ua in user_agents.items():
        print(f"\n🌐 {navegador}:")
        print(f"   {ua}")

# Ejecutar análisis
analizar_headers_http()

## 3. Sesiones y Cookies: Manteniendo el Estado 🍪

Las sesiones HTTP permiten mantener el estado entre múltiples requests. Son esenciales para sitios que requieren login o tienen comportamiento dinámico.

In [ ]:
def demostrar_sesiones_cookies():
    """Demostración completa de sesiones y cookies"""
    print("🍪 SESIONES Y COOKIES EN WEB SCRAPING\n")
    print("═" * 70)
    
    # 1. Sesiones básicas
    print("1️⃣ Creación y Uso de Sesiones")
    print("-" * 40)
    
    # Crear una sesión
    session = requests.Session()
    
    # Configurar headers por defecto para la sesión
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept-Language': 'es-ES,es;q=0.9'
    })
    
    print("✅ Sesión creada con headers por defecto")
    print(f"📋 Headers de sesión: {len(session.headers)} configurados")
    
    # 2. Manejo automático de cookies
    print("\n2️⃣ Manejo Automático de Cookies")
    print("-" * 40)
    
    # Primera request - establecer cookies
    response = session.get('https://httpbin.org/cookies/set/session_id/abc123')
    print(f"✅ Primera request completada: {response.status_code}")
    
    # Ver cookies en la sesión
    print(f"🍪 Cookies almacenadas en sesión: {len(session.cookies)}")
    for cookie in session.cookies:
        print(f"   {cookie.name}: {cookie.value} (domain: {cookie.domain})")
    
    # Segunda request - cookies se envían automáticamente
    response = session.get('https://httpbin.org/cookies')
    data = response.json()
    print(f"\n📤 Cookies enviadas automáticamente: {data['cookies']}")
    
    # 3. Manipulación manual de cookies
    print("\n3️⃣ Manipulación Manual de Cookies")
    print("-" * 40)
    
    # Añadir cookies manualmente
    session.cookies.set('custom_cookie', 'valor_personalizado', domain='httpbin.org')
    session.cookies.set('tracking_id', 'usuario_12345', domain='httpbin.org')
    
    # Verificar cookies
    response = session.get('https://httpbin.org/cookies')
    data = response.json()
    print(f"🍪 Todas las cookies: {data['cookies']}")
    
    # 4. Persistencia de cookies
    print("\n4️⃣ Persistencia de Cookies")
    print("-" * 40)
    
    import pickle
    import os
    
    # Guardar cookies
    cookies_file = '../data/cookies_session.pkl'
    os.makedirs(os.path.dirname(cookies_file), exist_ok=True)
    
    with open(cookies_file, 'wb') as f:
        pickle.dump(session.cookies, f)
    print(f"💾 Cookies guardadas en: {cookies_file}")
    
    # Crear nueva sesión y cargar cookies
    nueva_session = requests.Session()
    
    with open(cookies_file, 'rb') as f:
        nueva_session.cookies.update(pickle.load(f))
    
    print(f"📂 Cookies cargadas en nueva sesión: {len(nueva_session.cookies)}")
    
    # Verificar que funcionan
    response = nueva_session.get('https://httpbin.org/cookies')
    data = response.json()
    print(f"✅ Cookies persistentes funcionando: {len(data['cookies'])} cookies activas")
    
    # 5. Cookies con expiración
    print("\n5️⃣ Análisis de Cookies con Expiración")
    print("-" * 40)
    
    # Obtener un sitio real con cookies complejas
    try:
        response = session.get('https://httpbin.org/cookies/set/expires_cookie/temporal', timeout=5)
        
        print("🕐 Información detallada de cookies:")
        for cookie in session.cookies:
            print(f"\n   📋 Cookie: {cookie.name}")
            print(f"      Valor: {cookie.value}")
            print(f"      Dominio: {cookie.domain}")
            print(f"      Ruta: {cookie.path}")
            print(f"      Segura: {cookie.secure}")
            print(f"      HTTPOnly: {cookie.has_nonstandard_attr('HttpOnly')}")
            if cookie.expires:
                import datetime
                expiry = datetime.datetime.fromtimestamp(cookie.expires)
                print(f"      Expira: {expiry}")
            else:
                print(f"      Expira: Sesión (no persistente)")
                
    except requests.RequestException as e:
        print(f"❌ Error al obtener cookies detalladas: {e}")
    
    return session

# Ejecutar demostración
mi_session = demostrar_sesiones_cookies()
print(f"\n🎉 Sesión configurada y lista para usar con {len(mi_session.cookies)} cookies")

## 4. Formularios y POST Requests 📝

Muchos sitios web requieren interactuar con formularios para acceder al contenido. Aprender a manejar formularios es crucial para el scraping avanzado.

In [ ]:
from bs4 import BeautifulSoup
import urllib.parse

def manejar_formularios():
    """Demostración completa de manejo de formularios"""
    print("📝 MANEJO AVANZADO DE FORMULARIOS\n")
    print("═" * 70)
    
    # 1. Análisis de formularios HTML
    print("1️⃣ Análisis de Formularios HTML")
    print("-" * 40)
    
    # HTML de formulario complejo
    formulario_html = """
    <form method="post" action="/login" enctype="application/x-www-form-urlencoded">
        <input type="hidden" name="csrf_token" value="abc123xyz789" />
        <input type="hidden" name="form_id" value="login_form" />
        
        <label for="username">Usuario:</label>
        <input type="text" name="username" id="username" required />
        
        <label for="password">Contraseña:</label>
        <input type="password" name="password" id="password" required />
        
        <label for="remember">Recordarme:</label>
        <input type="checkbox" name="remember" id="remember" value="1" />
        
        <label for="role">Rol:</label>
        <select name="role" id="role">
            <option value="user" selected>Usuario</option>
            <option value="admin">Administrador</option>
        </select>
        
        <button type="submit">Iniciar Sesión</button>
    </form>
    """
    
    # Parsear formulario
    soup = BeautifulSoup(formulario_html, 'html.parser')
    form = soup.find('form')
    
    print(f"📋 Formulario encontrado:")
    print(f"   Método: {form.get('method', 'GET').upper()}")
    print(f"   Acción: {form.get('action', 'misma página')}")
    print(f"   Encoding: {form.get('enctype', 'application/x-www-form-urlencoded')}")
    
    # Extraer todos los campos
    campos = form.find_all(['input', 'select', 'textarea'])
    print(f"\n🔍 Campos encontrados: {len(campos)}")
    
    form_data = {}
    for campo in campos:
        name = campo.get('name')
        if not name:
            continue
            
        tipo = campo.get('type', campo.name)
        valor = campo.get('value', '')
        required = '✅' if campo.has_attr('required') else '❌'
        
        print(f"   📝 {name} ({tipo}): '{valor}' | Requerido: {required}")
        
        # Configurar valores por defecto
        if tipo == 'hidden':
            form_data[name] = valor
        elif tipo == 'checkbox' and not valor:
            form_data[name] = '1' if campo.has_attr('checked') else ''
        elif tipo == 'select':
            selected_option = campo.find('option', selected=True)
            form_data[name] = selected_option.get('value') if selected_option else ''
        else:
            form_data[name] = valor
    
    # 2. Simulación de envío de formulario
    print("\n2️⃣ Simulación de Envío de Formulario")
    print("-" * 40)
    
    # Completar formulario
    form_data.update({
        'username': 'usuario_scraper',
        'password': 'password123',
        'remember': '1',
        'role': 'user'
    })
    
    print("📤 Datos del formulario a enviar:")
    for key, value in form_data.items():
        print(f"   {key}: {value}")
    
    # Simular POST request
    session = requests.Session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Referer': 'https://httpbin.org/forms/post'
    }
    
    try:
        response = session.post('https://httpbin.org/post', data=form_data, headers=headers)
        
        if response.status_code == 200:
            print(f"\n✅ Formulario enviado exitosamente: {response.status_code}")
            
            data = response.json()
            print(f"📨 Datos recibidos por el servidor: {data['form']}")
            print(f"📋 Headers enviados: {len(data['headers'])}")
        else:
            print(f"❌ Error al enviar formulario: {response.status_code}")
            
    except requests.RequestException as e:
        print(f"❌ Error de conexión: {e}")
    
    # 3. Manejo de diferentes tipos de encoding
    print("\n3️⃣ Diferentes Tipos de Encoding")
    print("-" * 40)
    
    # application/x-www-form-urlencoded (por defecto)
    print("\n📝 Form URL Encoded:")
    response = session.post('https://httpbin.org/post', data=form_data)
    data = response.json()
    print(f"   Content-Type enviado: {data['headers'].get('Content-Type')}")
    print(f"   Datos: {data['form']}")
    
    # multipart/form-data (para archivos)
    print("\n📁 Multipart Form Data:")
    files_data = {
        'username': 'usuario_archivo',
        'file': ('test.txt', 'Contenido del archivo de prueba', 'text/plain')
    }
    
    response = session.post('https://httpbin.org/post', files=files_data)
    data = response.json()
    print(f"   Content-Type enviado: {data['headers'].get('Content-Type')[:50]}...")
    print(f"   Archivos: {data['files']}")
    print(f"   Form data: {data['form']}")
    
    # application/json
    print("\n📊 JSON Data:")
    json_data = {'username': 'usuario_json', 'preferences': {'theme': 'dark', 'lang': 'es'}}
    response = session.post('https://httpbin.org/post', json=json_data)
    data = response.json()
    print(f"   Content-Type enviado: {data['headers'].get('Content-Type')}")
    print(f"   JSON: {data['json']}")

# Ejecutar demostración
manejar_formularios()

## 5. Beautiful Soup Avanzado: Técnicas Profesionales 🍲

Beautiful Soup tiene características avanzadas que pueden hacer tu scraping mucho más eficiente y robusto.

In [ ]:
import re
from bs4 import BeautifulSoup, NavigableString, Tag
from bs4.builder import XMLParsedAsHTMLWarning
import warnings

# Suprimir warnings de Beautiful Soup para output más limpio
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

def beautiful_soup_avanzado():
    """Técnicas avanzadas de Beautiful Soup"""
    print("🍲 BEAUTIFUL SOUP: TÉCNICAS AVANZADAS\n")
    print("═" * 70)
    
    # HTML complejo para demostración
    html_complejo = """
    <!DOCTYPE html>
    <html lang="es">
    <head>
        <meta charset="UTF-8">
        <title>Página Compleja de Ejemplo</title>
        <script type="application/ld+json">
        {
            "@context": "http://schema.org",
            "@type": "Article",
            "headline": "Noticia Principal",
            "author": "Juan Pérez",
            "datePublished": "2024-01-15"
        }
        </script>
    </head>
    <body>
        <div class="container main-content" data-section="primary">
            <article id="article-123" class="post featured" data-id="123" data-category="tech">
                <header>
                    <h1>Título Principal con <em>énfasis</em> y <strong>importancia</strong></h1>
                    <time datetime="2024-01-15T10:30:00Z">15 de Enero, 2024</time>
                    <div class="tags">
                        <span class="tag tech">Tecnología</span>
                        <span class="tag ai">Inteligencia Artificial</span>
                        <span class="tag python">Python</span>
                    </div>
                </header>
                
                <div class="content">
                    <p class="intro">Este es el párrafo introductorio con <a href="#link1">enlace interno</a>.</p>
                    <p>Párrafo normal con texto y <code>código inline</code>.</p>
                    
                    <!-- Lista con diferentes tipos -->
                    <ul class="lista-items">
                        <li data-priority="high">Item importante</li>
                        <li data-priority="medium">Item normal</li>
                        <li data-priority="low">Item de baja prioridad</li>
                    </ul>
                    
                    <!-- Tabla de datos -->
                    <table class="data-table">
                        <thead>
                            <tr>
                                <th>Producto</th>
                                <th>Precio</th>
                                <th data-sortable="true">Rating</th>
                            </tr>
                        </thead>
                        <tbody>
                            <tr data-product-id="1">
                                <td>Producto A</td>
                                <td class="price">$99.99</td>
                                <td class="rating">4.5</td>
                            </tr>
                            <tr data-product-id="2">
                                <td>Producto B</td>
                                <td class="price">$149.99</td>
                                <td class="rating">4.8</td>
                            </tr>
                        </tbody>
                    </table>
                </div>
                
                <footer class="article-footer">
                    <div class="author">Por: <span class="author-name">María González</span></div>
                    <div class="social-sharing">
                        <a href="#" class="social-link" data-platform="twitter">Twitter</a>
                        <a href="#" class="social-link" data-platform="facebook">Facebook</a>
                    </div>
                </footer>
            </article>
            
            <!-- Comentarios -->
            <section id="comments" class="comments-section">
                <h2>Comentarios (3)</h2>
                <div class="comment" data-comment-id="1">
                    <strong>Usuario1</strong>: Excelente artículo!
                    <time>hace 2 horas</time>
                </div>
                <div class="comment" data-comment-id="2">
                    <strong>Usuario2</strong>: Muy informativo, gracias.
                    <time>hace 1 hora</time>
                </div>
            </section>
        </div>
        
        <!-- Sidebar -->
        <aside class="sidebar">
            <div class="widget recent-posts">
                <h3>Posts Recientes</h3>
                <ul>
                    <li><a href="/post/1">Post 1</a></li>
                    <li><a href="/post/2">Post 2</a></li>
                </ul>
            </div>
        </aside>
    </body>
    </html>
    """
    
    # Crear soup
    soup = BeautifulSoup(html_complejo, 'html.parser')
    
    # 1. Búsquedas con funciones personalizadas
    print("1️⃣ Búsquedas con Funciones Personalizadas")
    print("-" * 50)
    
    def tiene_precio_alto(tag):
        """Función personalizada para encontrar precios altos"""
        if tag.name != 'td' or not tag.has_attr('class'):
            return False
        if 'price' not in tag.get('class'):
            return False
        try:
            precio = float(tag.text.replace('$', ''))
            return precio > 100
        except ValueError:
            return False
    
    precios_altos = soup.find_all(tiene_precio_alto)
    print(f"💰 Precios altos encontrados: {len(precios_altos)}")
    for precio in precios_altos:
        producto = precio.parent.find('td').text
        print(f"   📦 {producto}: {precio.text}")
    
    # 2. Navegación compleja del árbol
    print("\n2️⃣ Navegación Compleja del Árbol")
    print("-" * 50)
    
    # Encontrar elemento y navegar por sus relaciones
    article = soup.find('article', id='article-123')
    
    print(f"📝 Artículo encontrado: ID {article.get('id')}")
    print(f"   👨‍👧‍👦 Padre: {article.parent.name} (clase: {article.parent.get('class')})")
    print(f"   👶 Hijos directos: {len([child for child in article.children if isinstance(child, Tag)])}")
    
    # Hermanos
    siguiente_hermano = article.find_next_sibling()
    if siguiente_hermano:
        print(f"   👫 Siguiente hermano: {siguiente_hermano.name} (ID: {siguiente_hermano.get('id')})")
    
    # Ancestros específicos
    container = article.find_parent('div', class_='container')
    print(f"   👴 Container padre: {container.get('class')} (data-section: {container.get('data-section')})")
    
    # 3. Extracción de texto avanzada
    print("\n3️⃣ Extracción de Texto Avanzada")
    print("-" * 50)
    
    titulo = soup.find('h1')
    
    # Diferentes formas de obtener texto
    print(f"📝 Texto completo: {titulo.get_text()}")
    print(f"📝 Texto con separador: {titulo.get_text(separator=' | ', strip=True)}")
    print(f"📝 Solo strings navegables: {[s.strip() for s in titulo.strings if s.strip()]}")
    
    # Texto de elementos específicos
    enfasis = titulo.find('em')
    importante = titulo.find('strong')
    print(f"📝 Texto con énfasis: {enfasis.text if enfasis else 'No encontrado'}")
    print(f"📝 Texto importante: {importante.text if importante else 'No encontrado'}")
    
    # 4. Expresiones regulares avanzadas
    print("\n4️⃣ Búsquedas con Expresiones Regulares")
    print("-" * 50)
    
    # Buscar por texto que contenga patrones
    comentarios_recientes = soup.find_all(string=re.compile(r'hace \d+ hora'))
    print(f"🕐 Comentarios recientes: {len(comentarios_recientes)}")
    for comentario in comentarios_recientes:
        print(f"   💬 {comentario.strip()}")
    
    # Buscar atributos con patrones
    elementos_data = soup.find_all(attrs={'data-comment-id': re.compile(r'\d+')})
    print(f"\n🏷️ Elementos con data-comment-id numérico: {len(elementos_data)}")
    for elem in elementos_data:
        id_comentario = elem.get('data-comment-id')
        usuario = elem.find('strong').text if elem.find('strong') else 'Usuario desconocido'
        print(f"   #{id_comentario}: {usuario}")
    
    # Buscar clases con patrones
    tags = soup.find_all('span', class_=re.compile(r'tag \w+'))
    print(f"\n🏷️ Tags encontrados: {len(tags)}")
    for tag in tags:
        clases = ' '.join(tag.get('class'))
        print(f"   🏷️ {tag.text}: {clases}")
    
    # 5. Modificación del DOM
    print("\n5️⃣ Modificación del DOM")
    print("-" * 50)
    
    # Crear copia para modificar
    soup_copia = BeautifulSoup(str(soup), 'html.parser')
    
    # Añadir elemento
    nuevo_comentario = soup_copia.new_tag('div', **{'class': 'comment', 'data-comment-id': '3'})
    nuevo_comentario.string = 'Comentario añadido por scraper'
    
    seccion_comentarios = soup_copia.find('section', id='comments')
    seccion_comentarios.append(nuevo_comentario)
    
    print(f"➕ Comentario añadido. Total comentarios: {len(soup_copia.find_all('div', class_='comment'))}")
    
    # Modificar atributo
    article_copia = soup_copia.find('article')
    article_copia['data-processed'] = 'true'
    article_copia['data-scraped-at'] = '2024-01-15'
    
    print(f"🔄 Atributos modificados: {article_copia.get('data-processed')} | {article_copia.get('data-scraped-at')}")
    
    # Eliminar elementos
    scripts = soup_copia.find_all('script')
    print(f"🗑️ Scripts encontrados para eliminar: {len(scripts)}")
    for script in scripts:
        script.decompose()
    
    scripts_despues = soup_copia.find_all('script')
    print(f"✅ Scripts después de eliminar: {len(scripts_despues)}")
    
    # 6. Extracción de JSON-LD (datos estructurados)
    print("\n6️⃣ Extracción de Datos Estructurados JSON-LD")
    print("-" * 50)
    
    import json
    
    # Buscar scripts con datos estructurados
    json_scripts = soup.find_all('script', type='application/ld+json')
    print(f"📊 Scripts JSON-LD encontrados: {len(json_scripts)}")
    
    for i, script in enumerate(json_scripts, 1):
        try:
            data = json.loads(script.string)
            print(f"\n   📄 JSON-LD #{i}:")
            print(f"      Tipo: {data.get('@type')}")
            print(f"      Título: {data.get('headline')}")
            print(f"      Autor: {data.get('author')}")
            print(f"      Fecha: {data.get('datePublished')}")
        except json.JSONDecodeError as e:
            print(f"   ❌ Error parsing JSON-LD: {e}")

# Ejecutar demostración
beautiful_soup_avanzado()

## 6. Manejo de Encodings y Contenido Internacional 🌍

Los problemas de encoding son comunes en web scraping, especialmente con contenido en diferentes idiomas.

In [ ]:
import chardet
from bs4 import BeautifulSoup

def manejar_encodings():
    """Manejo avanzado de encodings y contenido internacional"""
    print("🌍 MANEJO DE ENCODINGS Y CONTENIDO INTERNACIONAL\n")
    print("═" * 70)
    
    # 1. Detección automática de encoding
    print("1️⃣ Detección Automática de Encoding")
    print("-" * 50)
    
    # Simular contenido con diferentes encodings
    textos_ejemplo = {
        'utf-8': 'Hola mundo! Café, niño, corazón 💖',
        'latin-1': 'Hola mundo! Caf\xe9, ni\xf1o, coraz\xf3n',
        'cp1252': 'Hola mundo! Smart quotes: \u201cHello\u201d',
        'ascii': 'Hello world! Basic ASCII text only'
    }
    
    for encoding, texto in textos_ejemplo.items():
        try:
            # Simular bytes con encoding específico
            if encoding == 'latin-1':
                # Contenido que parece latin-1
                texto_limpio = 'Hola mundo! Café, niño, corazón'
                bytes_contenido = texto_limpio.encode('latin-1')
            elif encoding == 'cp1252':
                texto_limpio = 'Hola mundo! Smart quotes: "Hello"'
                bytes_contenido = texto_limpio.encode('cp1252')
            else:
                bytes_contenido = texto.encode(encoding)
            
            # Detectar encoding
            detected = chardet.detect(bytes_contenido)
            
            print(f"\n📝 Encoding {encoding}:")
            print(f"   Detectado: {detected['encoding']} (confianza: {detected['confidence']:.2f})")
            print(f"   Texto original: {texto}")
            
            # Decodificar correctamente
            if detected['encoding']:
                texto_decodificado = bytes_contenido.decode(detected['encoding'])
                print(f"   Decodificado: {texto_decodificado}")
            
        except UnicodeError as e:
            print(f"   ❌ Error de Unicode: {e}")
    
    # 2. Función robusta para detección de encoding
    print("\n\n2️⃣ Función Robusta de Detección")
    print("-" * 50)
    
    def detectar_encoding_robusto(response):
        """Detecta encoding de forma robusta"""
        
        # 1. Intentar con el encoding del header
        content_type = response.headers.get('content-type', '')
        if 'charset=' in content_type:
            charset = content_type.split('charset=')[-1].split(';')[0].strip()
            print(f"   📋 Encoding del header: {charset}")
            try:
                test_decode = response.content[:100].decode(charset)
                return charset, 'header'
            except UnicodeDecodeError:
                print(f"   ⚠️ Header encoding {charset} falló")
        
        # 2. Buscar en meta tags
        try:
            # Parsear solo el head con encoding aproximado
            head_content = response.content[:2000]  # Primeros 2KB
            soup_temp = BeautifulSoup(head_content, 'html.parser')
            
            # Buscar meta charset
            meta_charset = soup_temp.find('meta', charset=True)
            if meta_charset:
                charset = meta_charset.get('charset')
                print(f"   📋 Meta charset: {charset}")
                try:
                    test_decode = response.content[:100].decode(charset)
                    return charset, 'meta_charset'
                except UnicodeDecodeError:
                    pass
            
            # Buscar meta http-equiv
            meta_equiv = soup_temp.find('meta', attrs={'http-equiv': 'Content-Type'})
            if meta_equiv and meta_equiv.get('content'):
                content = meta_equiv.get('content')
                if 'charset=' in content:
                    charset = content.split('charset=')[-1].strip()
                    print(f"   📋 Meta http-equiv: {charset}")
                    try:
                        test_decode = response.content[:100].decode(charset)
                        return charset, 'meta_equiv'
                    except UnicodeDecodeError:
                        pass
                        
        except Exception as e:
            print(f"   ⚠️ Error parseando meta tags: {e}")
        
        # 3. Usar chardet como último recurso
        detected = chardet.detect(response.content[:10000])  # Primeros 10KB
        if detected['encoding'] and detected['confidence'] > 0.7:
            print(f"   🔍 Chardet detectó: {detected['encoding']} (confianza: {detected['confidence']:.2f})")
            return detected['encoding'], 'chardet'
        
        # 4. UTF-8 como fallback
        print(f"   🔄 Usando UTF-8 como fallback")
        return 'utf-8', 'fallback'
    
    # Probar con diferentes sitios
    sitios_prueba = [
        'https://httpbin.org/html',
        'https://httpbin.org/encoding/utf8'
    ]
    
    session = requests.Session()
    
    for url in sitios_prueba:
        try:
            print(f"\n🌐 Probando: {url}")
            response = session.get(url, timeout=5)
            
            encoding, metodo = detectar_encoding_robusto(response)
            print(f"   ✅ Encoding final: {encoding} (método: {metodo})")
            
            # Decodificar contenido
            try:
                if encoding != response.encoding:
                    response.encoding = encoding
                
                contenido = response.text[:100]
                print(f"   📝 Contenido (primeros 100 chars): {contenido[:50]}...")
                
            except UnicodeDecodeError as e:
                print(f"   ❌ Error decodificando: {e}")
                
        except requests.RequestException as e:
            print(f"   ❌ Error de conexión: {e}")
    
    # 3. Manejo de caracteres especiales
    print("\n\n3️⃣ Manejo de Caracteres Especiales")
    print("-" * 50)
    
    # HTML con caracteres especiales
    html_internacional = """
    <!DOCTYPE html>
    <html>
    <head><meta charset="UTF-8"></head>
    <body>
        <h1>Texto Internacional</h1>
        <p>Español: niño, corazón, ñoño</p>
        <p>Francés: café, résumé, naïve</p>
        <p>Alemán: Müller, Größe, weiß</p>
        <p>Japonés: こんにちは (Konnichiwa)</p>
        <p>Emojis: 🌟 💖 🚀 🍕</p>
        <p>Símbolos: © ® ™ € £ ¥</p>
        <p>Entidades HTML: &lt;tag&gt; &amp; &quot;quote&quot;</p>
    </body>
    </html>
    """
    
    soup = BeautifulSoup(html_internacional, 'html.parser')
    
    print("🌍 Extracción de texto internacional:")
    parrafos = soup.find_all('p')
    
    for i, p in enumerate(parrafos, 1):
        texto = p.get_text()
        print(f"   {i}. {texto}")
        
        # Mostrar información de encoding
        try:
            bytes_utf8 = texto.encode('utf-8')
            print(f"      UTF-8 bytes: {len(bytes_utf8)} bytes")
        except UnicodeEncodeError as e:
            print(f"      ❌ Error encoding UTF-8: {e}")
    
    # 4. Limpieza de texto
    print("\n\n4️⃣ Limpieza de Texto Avanzada")
    print("-" * 50)
    
    import unicodedata
    
    def limpiar_texto_avanzado(texto):
        """Limpia texto de forma avanzada"""
        
        # 1. Normalizar Unicode
        texto = unicodedata.normalize('NFKD', texto)
        
        # 2. Limpiar espacios en blanco
        texto = ' '.join(texto.split())
        
        # 3. Remover caracteres de control
        texto = ''.join(char for char in texto if unicodedata.category(char)[0] != 'C' or char in '\n\t\r')
        
        return texto.strip()
    
    # Texto problemático para limpiar
    texto_sucio = "\n\n  Texto    con    espacios\t\t\textra   \n  y caracteres\x00 de control\x01  \n\n"
    
    print(f"📝 Texto original: {repr(texto_sucio)}")
    texto_limpio = limpiar_texto_avanzado(texto_sucio)
    print(f"✨ Texto limpio: {repr(texto_limpio)}")

# Instalar chardet si no está disponible
try:
    import chardet
    manejar_encodings()
except ImportError:
    print("📦 Instalando chardet...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'chardet'])
    import chardet
    manejar_encodings()

## 7. Paginación y Crawling Avanzado 📄

Muchos sitios web dividen el contenido en páginas. Aprender a manejar paginación es crucial para extraer datasets completos.

In [ ]:
import time
import random
from urllib.parse import urljoin, urlparse, parse_qs

def manejar_paginacion():
    """Técnicas avanzadas para manejar paginación"""
    print("📄 MANEJO AVANZADO DE PAGINACIÓN\n")
    print("═" * 70)
    
    # 1. Paginación básica con quotes.toscrape.com
    print("1️⃣ Paginación Básica")
    print("-" * 40)
    
    def scrape_quotes_paginado():
        """Scraper con paginación automática"""
        base_url = "http://quotes.toscrape.com"
        session = requests.Session()
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        
        todas_citas = []
        pagina = 1
        max_paginas = 3  # Limitar para demostración
        
        while pagina <= max_paginas:
            url = f"{base_url}/page/{pagina}/"
            print(f"\n📖 Procesando página {pagina}: {url}")
            
            try:
                response = session.get(url, timeout=10)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Extraer citas de esta página
                quotes = soup.find_all('div', class_='quote')
                
                if not quotes:
                    print(f"   ❌ No se encontraron citas en página {pagina}. Terminando.")
                    break
                
                print(f"   ✅ Encontradas {len(quotes)} citas")
                
                for quote in quotes:
                    texto = quote.find('span', class_='text').text
                    autor = quote.find('small', class_='author').text
                    tags = [tag.text for tag in quote.find_all('a', class_='tag')]
                    
                    todas_citas.append({
                        'pagina': pagina,
                        'texto': texto,
                        'autor': autor,
                        'tags': tags
                    })
                
                # Buscar enlace a siguiente página
                next_btn = soup.find('li', class_='next')
                if not next_btn:
                    print(f"   🏁 No hay más páginas después de la página {pagina}")
                    break
                
                # Delay entre requests
                delay = random.uniform(1, 2)
                print(f"   ⏳ Esperando {delay:.2f} segundos...")
                time.sleep(delay)
                
                pagina += 1
                
            except requests.RequestException as e:
                print(f"   ❌ Error en página {pagina}: {e}")
                break
        
        return todas_citas
    
    # Ejecutar scraping paginado
    citas_paginadas = scrape_quotes_paginado()
    
    print(f"\n📊 Resumen de paginación:")
    print(f"   📚 Total de citas recopiladas: {len(citas_paginadas)}")
    
    # Agrupar por página
    from collections import Counter
    citas_por_pagina = Counter(cita['pagina'] for cita in citas_paginadas)
    for pagina, cantidad in sorted(citas_por_pagina.items()):
        print(f"   📄 Página {pagina}: {cantidad} citas")
    
    # Autores más frecuentes
    autores_frecuentes = Counter(cita['autor'] for cita in citas_paginadas)
    print(f"\n👥 Autores más frecuentes:")
    for autor, cantidad in autores_frecuentes.most_common(3):
        print(f"   ✍️ {autor}: {cantidad} citas")
    
    # 2. Detección automática de paginación
    print("\n\n2️⃣ Detección Automática de Paginación")
    print("-" * 50)
    
    def detectar_paginacion(soup, current_url):
        """Detecta patrones de paginación automáticamente"""
        
        patrones_paginacion = []
        
        # 1. Buscar enlaces "Next" típicos
        next_patterns = [
            ('a', {'class': re.compile(r'next', re.I)}),
            ('a', {'id': re.compile(r'next', re.I)}),
            ('a', {'text': re.compile(r'next|siguiente|→|»', re.I)}),
            ('li', {'class': re.compile(r'next', re.I)})
        ]
        
        for tag, attrs in next_patterns:
            if 'text' in attrs:
                elementos = soup.find_all(tag, string=attrs['text'])
            else:
                elementos = soup.find_all(tag, attrs)
            
            for elem in elementos:
                # Si es li, buscar enlace dentro
                if tag == 'li':
                    link = elem.find('a')
                    if link and link.get('href'):
                        href = link.get('href')
                        full_url = urljoin(current_url, href)
                        patrones_paginacion.append({
                            'tipo': 'next_link_li',
                            'elemento': str(elem)[:100] + '...',
                            'url': full_url
                        })
                else:
                    href = elem.get('href')
                    if href:
                        full_url = urljoin(current_url, href)
                        patrones_paginacion.append({
                            'tipo': 'next_link_direct',
                            'elemento': str(elem)[:100] + '...',
                            'url': full_url
                        })
        
        # 2. Buscar enlaces numerados (1, 2, 3, ...)
        links_numericos = soup.find_all('a', href=True)
        for link in links_numericos:
            href = link.get('href')
            texto = link.get_text(strip=True)
            
            if texto.isdigit() and int(texto) > 1:
                full_url = urljoin(current_url, href)
                patrones_paginacion.append({
                    'tipo': 'numbered_page',
                    'elemento': str(link),
                    'url': full_url,
                    'numero_pagina': int(texto)
                })
        
        # 3. Buscar patrones en URLs
        parsed_url = urlparse(current_url)
        path_parts = parsed_url.path.strip('/').split('/')
        query_params = parse_qs(parsed_url.query)
        
        url_info = {
            'base_url': f"{parsed_url.scheme}://{parsed_url.netloc}",
            'path_parts': path_parts,
            'query_params': query_params
        }
        
        return patrones_paginacion, url_info
    
    # Probar detección con quotes.toscrape.com
    try:
        session = requests.Session()
        response = session.get('http://quotes.toscrape.com/page/1/', timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        patrones, url_info = detectar_paginacion(soup, response.url)
        
        print(f"🔍 Patrones de paginación detectados: {len(patrones)}")
        for i, patron in enumerate(patrones[:5], 1):  # Mostrar primeros 5
            print(f"\n   {i}. Tipo: {patron['tipo']}")
            if 'numero_pagina' in patron:
                print(f"      Página: {patron['numero_pagina']}")
            print(f"      URL: {patron['url']}")
            print(f"      Elemento: {patron['elemento'][:80]}...")
        
        print(f"\n🔗 Información de URL:")
        print(f"   Base URL: {url_info['base_url']}")
        print(f"   Path parts: {url_info['path_parts']}")
        print(f"   Query params: {url_info['query_params']}")
        
    except requests.RequestException as e:
        print(f"❌ Error probando detección: {e}")
    
    # 3. Estrategias de paginación
    print("\n\n3️⃣ Estrategias de Paginación")
    print("-" * 50)
    
    estrategias = {
        '🔗 Siguiente/Anterior': {
            'descripcion': 'Enlaces "Next" y "Previous"',
            'ventajas': 'Simple, respeta la estructura del sitio',
            'desventajas': 'Lento, secuencial',
            'ejemplo': '/page/next/ o ?page=2'
        },
        '🔢 Páginas Numeradas': {
            'descripcion': 'Enlaces directos a páginas específicas',
            'ventajas': 'Permite paralelización, acceso aleatorio',
            'desventajas': 'Puede requerir conocer el total de páginas',
            'ejemplo': '/page/1/, /page/2/, /page/3/'
        },
        '📊 Offset/Limit': {
            'descripcion': 'Parámetros de desplazamiento',
            'ventajas': 'Flexible, común en APIs',
            'desventajas': 'Puede tener problemas con datos cambiantes',
            'ejemplo': '?offset=20&limit=10'
        },
        '🔄 Scroll Infinito': {
            'descripcion': 'Cargar más contenido vía AJAX',
            'ventajas': 'UX moderno, eficiente',
            'desventajas': 'Requiere JavaScript/Selenium',
            'ejemplo': 'Requests AJAX con cursores o timestamps'
        }
    }
    
    for estrategia, info in estrategias.items():
        print(f"\n{estrategia}")
        print(f"   📋 {info['descripcion']}")
        print(f"   ✅ Ventajas: {info['ventajas']}")
        print(f"   ❌ Desventajas: {info['desventajas']}")
        print(f"   💡 Ejemplo: {info['ejemplo']}")

# Ejecutar demostración
manejar_paginacion()

## 8. Ejercicio Práctico Avanzado: Scraper de E-commerce 🛒

¡Hora de aplicar todo lo aprendido! Vamos a crear un scraper completo para un sitio de e-commerce con múltiples páginas.

In [ ]:
# EJERCICIO AVANZADO: Scraper de E-commerce con Paginación
# Objetivo: Crear un scraper completo que maneje sesiones, paginación y datos complejos

import json
import csv
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import List, Optional
import os

@dataclass
class Producto:
    """Clase para representar un producto"""
    id: str
    titulo: str
    precio: float
    precio_original: Optional[float]
    disponible: bool
    rating: float
    num_reviews: int
    imagen_url: str
    producto_url: str
    categoria: str
    fecha_scraping: str

class EcommerceScraper:
    """Scraper avanzado para sitios de e-commerce"""
    
    def __init__(self, base_url: str, max_paginas: int = 5):
        self.base_url = base_url
        self.max_paginas = max_paginas
        self.productos = []
        
        # Configurar sesión
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        })
        
        # Estadísticas
        self.stats = {
            'paginas_procesadas': 0,
            'productos_encontrados': 0,
            'errores': 0,
            'tiempo_inicio': None,
            'tiempo_total': 0
        }
    
    def extraer_producto(self, elemento_producto) -> Optional[Producto]:
        """Extrae información de un elemento producto"""
        try:
            # Extraer título
            titulo_elem = elemento_producto.find(['h2', 'h3', 'h4'], class_=re.compile(r'title|name|product-name', re.I))
            if not titulo_elem:
                titulo_elem = elemento_producto.find(['h2', 'h3', 'h4'])
            titulo = titulo_elem.get_text(strip=True) if titulo_elem else 'Título no encontrado'
            
            # Extraer precio
            precio_elem = elemento_producto.find(class_=re.compile(r'price|cost|amount', re.I))
            if precio_elem:
                precio_texto = precio_elem.get_text(strip=True)
                # Extraer número del precio
                precio_match = re.search(r'([\d,.]+)', precio_texto.replace('£', '').replace('$', ''))
                precio = float(precio_match.group(1).replace(',', '')) if precio_match else 0.0
            else:
                precio = 0.0
            
            # Extraer rating (si existe)
            rating_elem = elemento_producto.find(class_=re.compile(r'rating|star', re.I))
            rating = 0.0
            if rating_elem:
                # Buscar texto con rating
                rating_texto = rating_elem.get_text()
                rating_match = re.search(r'([\d.]+)', rating_texto)
                if rating_match:
                    rating = float(rating_match.group(1))
                # O buscar en clases CSS (ej: star-rating-4)
                else:
                    for clase in rating_elem.get('class', []):
                        rating_match = re.search(r'(\d)', clase)
                        if rating_match:
                            rating = float(rating_match.group(1))
                            break
            
            # Extraer imagen
            img_elem = elemento_producto.find('img')
            imagen_url = ''
            if img_elem:
                imagen_url = img_elem.get('src') or img_elem.get('data-src', '')
                if imagen_url and not imagen_url.startswith('http'):
                    imagen_url = urljoin(self.base_url, imagen_url)
            
            # Extraer enlace al producto
            link_elem = elemento_producto.find('a', href=True)
            producto_url = ''
            if link_elem:
                producto_url = link_elem.get('href')
                if producto_url and not producto_url.startswith('http'):
                    producto_url = urljoin(self.base_url, producto_url)
            
            # Generar ID único
            producto_id = str(hash(titulo + str(precio)))[-8:]
            
            return Producto(
                id=producto_id,
                titulo=titulo[:200],  # Limitar longitud
                precio=precio,
                precio_original=None,  # Podría implementarse
                disponible=True,  # Asumir disponible si aparece en lista
                rating=rating,
                num_reviews=0,  # Podría implementarse
                imagen_url=imagen_url,
                producto_url=producto_url,
                categoria='general',  # Podría extraerse de breadcrumbs
                fecha_scraping=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            )
            
        except Exception as e:
            print(f"   ❌ Error extrayendo producto: {e}")
            return None
    
    def scrape_pagina(self, url: str) -> List[Producto]:
        """Scrape una página específica"""
        productos_pagina = []
        
        try:
            print(f"\n🔍 Scraping: {url}")
            
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Buscar contenedores de productos con diferentes patrones
            selectores_producto = [
                'article.product',
                '.product-item',
                '.product-card',
                '.book',  # Para books.toscrape.com
                '.product',
                '[data-product-id]'
            ]
            
            elementos_productos = []
            for selector in selectores_producto:
                elementos = soup.select(selector)
                if elementos:
                    elementos_productos = elementos
                    print(f"   ✅ Usando selector: {selector} ({len(elementos)} productos)")
                    break
            
            if not elementos_productos:
                print(f"   ⚠️ No se encontraron productos con los selectores conocidos")
                return productos_pagina
            
            # Extraer cada producto
            for i, elemento in enumerate(elementos_productos, 1):
                producto = self.extraer_producto(elemento)
                if producto:
                    productos_pagina.append(producto)
                    print(f"   📦 Producto {i}: {producto.titulo[:50]}... - ${producto.precio}")
            
            print(f"   ✅ Extraídos {len(productos_pagina)} productos de esta página")
            
        except requests.RequestException as e:
            print(f"   ❌ Error de red: {e}")
            self.stats['errores'] += 1
        except Exception as e:
            print(f"   ❌ Error inesperado: {e}")
            self.stats['errores'] += 1
        
        return productos_pagina
    
    def encontrar_siguiente_pagina(self, soup, current_url: str) -> Optional[str]:
        """Encuentra la URL de la siguiente página"""
        
        # Patrones comunes para "siguiente página"
        patrones_next = [
            ('a', {'class': re.compile(r'next', re.I)}),
            ('li', {'class': 'next'}),
            ('a', {'title': re.compile(r'next', re.I)}),
            ('a', {'rel': 'next'})
        ]
        
        for tag, attrs in patrones_next:
            elemento = soup.find(tag, attrs)
            if elemento:
                if tag == 'li':
                    link = elemento.find('a')
                    if link and link.get('href'):
                        return urljoin(current_url, link.get('href'))
                else:
                    href = elemento.get('href')
                    if href:
                        return urljoin(current_url, href)
        
        return None
    
    def scrape_completo(self) -> List[Producto]:
        """Ejecuta el scraping completo con paginación"""
        print("🛒 INICIANDO SCRAPING DE E-COMMERCE\n")
        print("═" * 70)
        
        self.stats['tiempo_inicio'] = time.time()
        
        url_actual = self.base_url
        pagina_num = 1
        
        while pagina_num <= self.max_paginas and url_actual:
            print(f"\n📄 PÁGINA {pagina_num} de {self.max_paginas}")
            print(f"🔗 URL: {url_actual}")
            
            # Scrape página actual
            productos_pagina = self.scrape_pagina(url_actual)
            self.productos.extend(productos_pagina)
            
            self.stats['paginas_procesadas'] += 1
            self.stats['productos_encontrados'] += len(productos_pagina)
            
            # Buscar siguiente página
            if pagina_num < self.max_paginas:
                try:
                    response = self.session.get(url_actual, timeout=10)
                    soup = BeautifulSoup(response.content, 'html.parser')
                    url_siguiente = self.encontrar_siguiente_pagina(soup, url_actual)
                    
                    if url_siguiente and url_siguiente != url_actual:
                        print(f"   ➡️ Siguiente página encontrada: {url_siguiente}")
                        url_actual = url_siguiente
                    else:
                        print(f"   🏁 No hay más páginas")
                        break
                        
                except Exception as e:
                    print(f"   ❌ Error buscando siguiente página: {e}")
                    break
            
            # Delay entre páginas
            if pagina_num < self.max_paginas and url_actual:
                delay = random.uniform(1, 3)
                print(f"   ⏳ Delay de {delay:.2f} segundos...")
                time.sleep(delay)
            
            pagina_num += 1
        
        self.stats['tiempo_total'] = time.time() - self.stats['tiempo_inicio']
        
        return self.productos
    
    def mostrar_estadisticas(self):
        """Muestra estadísticas del scraping"""
        print("\n" + "═" * 70)
        print("📊 ESTADÍSTICAS FINALES")
        print("═" * 70)
        
        print(f"📄 Páginas procesadas: {self.stats['paginas_procesadas']}")
        print(f"🛍️ Productos encontrados: {self.stats['productos_encontrados']}")
        print(f"❌ Errores: {self.stats['errores']}")
        print(f"⏱️ Tiempo total: {self.stats['tiempo_total']:.2f} segundos")
        
        if self.productos:
            precios = [p.precio for p in self.productos if p.precio > 0]
            if precios:
                print(f"💰 Precio promedio: ${sum(precios)/len(precios):.2f}")
                print(f"💰 Precio mínimo: ${min(precios):.2f}")
                print(f"💰 Precio máximo: ${max(precios):.2f}")
            
            # Top productos por precio
            productos_ordenados = sorted([p for p in self.productos if p.precio > 0], 
                                       key=lambda x: x.precio, reverse=True)
            
            print(f"\n🏆 TOP 3 PRODUCTOS MÁS CAROS:")
            for i, producto in enumerate(productos_ordenados[:3], 1):
                print(f"   {i}. {producto.titulo[:50]}... - ${producto.precio}")
    
    def guardar_datos(self, formato='json'):
        """Guarda los datos en diferentes formatos"""
        if not self.productos:
            print("❌ No hay productos para guardar")
            return
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Crear directorio de datos
        os.makedirs('../data', exist_ok=True)
        
        if formato == 'json':
            filename = f'../data/productos_{timestamp}.json'
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump([asdict(p) for p in self.productos], f, 
                         ensure_ascii=False, indent=2)
            print(f"💾 Datos guardados en JSON: {filename}")
        
        elif formato == 'csv':
            filename = f'../data/productos_{timestamp}.csv'
            with open(filename, 'w', newline='', encoding='utf-8') as f:
                if self.productos:
                    writer = csv.DictWriter(f, fieldnames=asdict(self.productos[0]).keys())
                    writer.writeheader()
                    for producto in self.productos:
                        writer.writerow(asdict(producto))
            print(f"📊 Datos guardados en CSV: {filename}")

# ¡EJECUTAR EL SCRAPER COMPLETO!
print("🚀 EJERCICIO AVANZADO: SCRAPER DE E-COMMERCE\n")
print("═" * 70)
print("Vamos a scrapear books.toscrape.com como ejemplo de e-commerce")
print("Este scraper incluye:")
print("  ✅ Manejo de sesiones HTTP")
print("  ✅ Paginación automática")
print("  ✅ Extracción de datos complejos")
print("  ✅ Manejo de errores robusto")
print("  ✅ Estadísticas detalladas")
print("  ✅ Exportación de datos")

# Crear y ejecutar scraper
scraper = EcommerceScraper(
    base_url='http://books.toscrape.com/',
    max_paginas=3  # Limitar para demostración
)

# Ejecutar scraping
productos = scraper.scrape_completo()

# Mostrar estadísticas
scraper.mostrar_estadisticas()

# Guardar datos
scraper.guardar_datos('json')
scraper.guardar_datos('csv')

print("\n🎉 ¡EJERCICIO COMPLETADO EXITOSAMENTE!")
print(f"📊 Se scrapearon {len(productos)} productos de {scraper.stats['paginas_procesadas']} páginas")
print("💾 Datos guardados en formato JSON y CSV")
print("\n🏆 ¡Has creado un scraper de e-commerce profesional!")

## 9. Resumen y Próximos Pasos 🎯

### 🎓 Lo que has dominado en esta lección:

#### ✅ **HTTP Avanzado**
- 🌐 Métodos HTTP y sus usos apropiados
- 🕵️ Headers HTTP y su importancia
- 🍪 Manejo avanzado de sesiones y cookies
- 📤 POST requests y envío de formularios

#### ✅ **Beautiful Soup Profesional**
- 🔍 Técnicas de búsqueda avanzadas
- 🌳 Navegación compleja del DOM
- 📝 Extracción de texto sofisticada
- 🛠️ Modificación del DOM

#### ✅ **Contenido Complejo**
- 🌍 Manejo de encodings internacionales
- 📄 Paginación y crawling avanzado
- 🔗 Detección automática de patrones
- 📊 Extracción de datos estructurados

#### ✅ **Proyecto Profesional**
- 🛒 Scraper de e-commerce completo
- 📊 Manejo de datos con dataclasses
- 💾 Exportación en múltiples formatos
- 📈 Estadísticas y monitoreo

### 🚀 Próxima Lección: XPath y Selección Avanzada

En la **Lección 3** exploraremos:

#### 🛤️ **XPath Completo**
- Sintaxis XPath desde básico hasta experto
- Funciones XPath avanzadas
- Comparación XPath vs CSS Selectors
- Casos de uso específicos

#### 🎯 **Selección Avanzada**
- Técnicas de selección precisas
- Manejo de contenido dinámico
- Optimización de selectores
- Debugging y troubleshooting

### 💪 Ejercicios para Practicar

1. **Scraper de Noticias con Login**: Crear un scraper que maneje autenticación
2. **Monitor de Precios**: Sistema que rastrea cambios de precios
3. **Agregador Multi-sitio**: Combinar datos de múltiples fuentes

### 📚 Recursos Recomendados

- [HTTP Status Codes Reference](https://httpstatuses.com/)
- [Beautiful Soup Advanced Guide](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Unicode in Python](https://docs.python.org/3/howto/unicode.html)
- [Regular Expressions Guide](https://docs.python.org/3/library/re.html)

---

### 🏆 ¡Felicidades por Completar la Lección 2!

Ahora dominas técnicas avanzadas de HTTP y Beautiful Soup. Puedes crear scrapers robustos que manejan:
- ✨ Sitios complejos con autenticación
- 🔄 Múltiples páginas automáticamente  
- 🌍 Contenido internacional
- 📊 Datos estructurados y complejos

**¡Tu arsenal de scraping se está volviendo muy poderoso! 🌟**